In [48]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [49]:
# For reproducibility
torch.manual_seed(1)

In [50]:
# Softmax
# Convert numbers to probabilities with softmax
z = torch.FloatTensor([1, 2, 3])

# PyTorch 가 가지고 있는 softmax function 을 이용
hypothesis = F.softmax(z, dim=0)
# 기존의 max 로 구하면 3 이 가장 큰 element 이므로 (0, 0, 1) 이지만
# softmax 함수는 입력받은 값을 0~1 사이의 값으로 모두 정규화한다.
# 가장 큰 값을 부여받은 클래스가 확률이 가장 높은 것으로 이용된다.
# 그리고 1, 2, 3 을 직접 이용하여 계산하는 것이 아닌
# exp(x) 를 이용한다. 즉 3 이 0.6652 = e^3/(e^1+e^2+e^3) 의 가장 높은 확률을 가진다.
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [51]:
# Softmax 의 값은 확률값이므로 모두 더하면 1 이다.
hypothesis.sum()

tensor(1.)

In [52]:
# Cross Entropy Loss (low-Level)
z = torch.rand(3, 5, requires_grad=True)      # 3 * 5 의 shape 로 0~1 사이의 랜덤한 값을 저장
hypothesis = F.softmax(z, dim=1)    # hypothesis의 두번째 차원에 대해 softmax 를 적용, 즉 한 행에 대해 모든 열의 element 를 더한 값은 1 이 된다.
print(hypothesis)
# prediction = y_hat
# classes = 5
# samples = 3

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [53]:
# 정답을 랜덤하게 생성
y = torch.randint(5, (3,)).long()     # 0~5 사이의 정수 중에 랜덤으로 3개를 뽑아 one hot vector의 index 값 생성
print(y)

tensor([0, 2, 1])


In [54]:
# One hot encoding
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(dim=1), 1)
# y 의 dim=1 에 대해 unsqueeze 를 하면 [0, 2, 1] -> [[0], [2], [1]] 이 되고
# 3 * 5 dim=1, 즉 두번째 차원(열)에 적용을 하여 0행의 0열, 1행의 2열, 2행의 1열 element 에 1 을 대입한다.

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [55]:
# Cost 계산
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
# y_one_hot, hypothesis 의 size 가 모두 3 * 5 로 동일하다.
# sum(dim=1) 에 의해 3 * 5 -> 3 * 1 이 되고,
# mean() 에 의해 3 * 1 -> scalar 가 된다.
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [56]:
# Cross-entropy Loss with torch.nn.functional

# Low level
torch.log(F.softmax(z, dim=1))
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean() # torch.log(F.softmax()) 대신에 F.log_softmax 를 사용할 수 있다.
# High level
F.log_softmax(z, dim=1)
F.nll_loss(F.log_softmax(z, dim=1), y)    # NLL = Negative Log Likelihood

# PyTorch also has F.cross_entropy (위의 F.log_softmax() 와 F.nll_loss() 를 결합한 기능을 한다.)
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [57]:
# Training with Low-level Cross Entropy Loss
# 임의의 데이터
x_train = [[1, 2, 1, 1],
           [1, 2, 1, 4],
           [1, 2, 2, 3],
           [1, 1, 1, 1],
           [1, 6, 7, 1],
           [1, 2, 1, 7],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]              # 8 개의 4차원 데이터
y_train = [2, 2, 2, 1, 1, 1, 0, 0]    # one hot encoding 을 하였을 때 1 이 있는 위치, 즉 index 값을 나타내므로 정수이다.
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [58]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)     # 4 * 3 이므로 classes = 3
b = torch.zeros(1, requires_grad=True)
# samples = 8, classes = 3, dimension = 4
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Cost 계산
  hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)  # or .mm or @
  y_one_hot = torch.zeros_like(hypothesis)               # hypothesis 와 동일한 size
  y_one_hot.scatter_(1, y_train.unsqueeze(dim=1), 1)
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

  # cost 로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100 번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.913857
Epoch  200/1000 Cost: 0.861137
Epoch  300/1000 Cost: 0.836729
Epoch  400/1000 Cost: 0.822129
Epoch  500/1000 Cost: 0.811892
Epoch  600/1000 Cost: 0.803786
Epoch  700/1000 Cost: 0.796772
Epoch  800/1000 Cost: 0.790341
Epoch  900/1000 Cost: 0.784257
Epoch 1000/1000 Cost: 0.778436


In [59]:
# Training with F.cross_entropy
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Cost 계산
  z = x_train.matmul(W) + b
  cost = F.cross_entropy(z, y_train)

  # cost 로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100 번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 cost: 1.098612
Epoch  100/1000 cost: 0.733060
Epoch  200/1000 cost: 0.628138
Epoch  300/1000 cost: 0.573433
Epoch  400/1000 cost: 0.539159
Epoch  500/1000 cost: 0.515472
Epoch  600/1000 cost: 0.498041
Epoch  700/1000 cost: 0.484628
Epoch  800/1000 cost: 0.473954
Epoch  900/1000 cost: 0.465232
Epoch 1000/1000 cost: 0.457952


In [60]:
# High-level Inplementation with nn.Module

class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(16, 7)

  def forward(self, x):
    return self.linear(x)

model = SoftmaxClassifierModel()

In [61]:
# Data

from google.colab import drive
drive.mount('/content/drive')

import numpy as np

xy = np.loadtxt('/content/drive/MyDrive/AI_class/Deep_learning_class/data-04-zoo.csv', delimiter=',', dtype=np.float32)

x_train = torch.FloatTensor(xy[:, 0:-1])
y_train = torch.LongTensor(xy[:, [-1]]).squeeze()

print(x_train.shape)    # shape 를 출력
print(len(x_train))     # 데이터의 개수를 출력
print(x_train[:5])      # 첫 5 개의 데이터를 출력

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
torch.Size([101, 16])
101
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [62]:
nb_classes = 7      # class 의 개수는 7
y_one_hot = torch.zeros((len(y_train), nb_classes))     # size = 100 * 7
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

In [63]:
# opimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  prediction = model(x_train)

  # Cost 계산
  cost = F.cross_entropy(prediction, y_train)

  # cost 로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100 번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.919160
Epoch  100/1000 Cost: 0.468405
Epoch  200/1000 Cost: 0.320585
Epoch  300/1000 Cost: 0.248953
Epoch  400/1000 Cost: 0.204819
Epoch  500/1000 Cost: 0.174506
Epoch  600/1000 Cost: 0.152248
Epoch  700/1000 Cost: 0.135139
Epoch  800/1000 Cost: 0.121543
Epoch  900/1000 Cost: 0.110461
Epoch 1000/1000 Cost: 0.101245
